In [ ]:
# connexion a la base de donnée spotify_db
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-1MKAJNIV\MSSQLSERVER02;'
                      'Database=spotify_db-test;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

# Creation  des tables qui vont accueillir les données
# Creation de la table artist
cursor.execute('''CREATE TABLE artist(   
                                    a_external_urls NVARCHAR(500),
                                    a_followers INT,
                                    a_id NVARCHAR(500) PRIMARY KEY,
                                    a_name NVARCHAR(500),
                                    a_popularity INT,
                                    a_uri NVARCHAR(500)
                                        )''')
# Creation de la table album
cursor.execute('''CREATE TABLE album(   
                                    al_type NVARCHAR(500),
                                    al_artists NVARCHAR(500),
                                    al_external_urls NVARCHAR(500),
                                    al_id NVARCHAR(500) PRIMARY KEY,
                                    al_images NVARCHAR(500),
                                    al_name NVARCHAR(500),
                                    al_release_date NVARCHAR(500),
                                    al_uri NVARCHAR(500)
                                        )''')
# Creation de la table track
cursor.execute('''CREATE TABLE track(   
                                    t_album  NVARCHAR(500) FOREIGN KEY REFERENCES album(al_id),
                                    t_artists NVARCHAR(500) FOREIGN KEY REFERENCES artist(a_id),
                                    t_disc_number INT,
                                    t_duration_ms INT,
                                    t_explicit BIT,
                                    t_external_urls NVARCHAR(500),
                                    t_id NVARCHAR(500) PRIMARY KEY,
                                    t_name NVARCHAR(500),
                                    t_popularity INT,
                                    t_preview_url NVARCHAR(500),
                                    t_number INT,
                                    t_uri NVARCHAR(500)
                                        )''')

conn.commit()

In [ ]:
import os
import spotipy
import spotipy.util as util

# Initialisation du token pour Spotify API 
# Enregistrement des Spotify credentials dans un dict user_config
user_config = {'username':'%%%','client_id':'%%%','client_secret':'%%%','redirect_uri':'http://localhost'}
# Creation du token
token = util.prompt_for_user_token(user_config['username'], 
                                   scope='playlist-read-private', 
                                   client_id=user_config['client_id'], 
                                   client_secret=user_config['client_secret'], 
                                   redirect_uri=user_config['redirect_uri'])
sp = spotipy.Spotify(auth=token)


In [ ]:
#initialisation des listes qui vont contenir les données sur les tracks
t_album = []
t_artists = []
t_disc_number = []
t_duration_ms = []
t_explicit = []
t_external_urls = []
t_id = []
t_name = []
t_popularity = []
t_preview_url = []
t_number = []
t_uri = []

#initialisation des listes qui vont contenir les données sur les albums
al_type = []
al_artists = []
al_external_urls = []
al_id = []
al_images = []
al_name = []
al_release_date = []
al_uri = []

#initialisation des listes qui vont contenir les données sur les artists
a_external_urls = []
a_followers = []
a_id = []
a_name = []
a_popularity = []
a_uri = []

In [ ]:
# requete (filtage des données qu'on veut récuperer)
q='year:2020'# les plus populaire en 2O2O

In [ ]:
# Recuperation des données sur artists,akbums,tarcks a partir de l'API search de Spotify 
#créait une boucle for imbriquée, augmentant le décalage de 50 dans la boucle externe jusqu'à ce que la limite maximale soit atteinte.
for i in range(0,2000,50):
    result = sp.search(q, type='artist,album,track', limit=50,offset=i)
    for i, a in enumerate(result['artists']['items']): #Parcour de l'objet Spotify pour recuper les données des artists 
        a_external_urls.append(a['external_urls']['spotify'])
        a_followers.append(a['followers']['total'])
        a_id.append(a['id'])
        a_name.append(a['name'])
        a_popularity.append(a['popularity'])
        a_uri.append(a['uri'])
    for i, al in enumerate(result['albums']['items']):# les données des albums
        al_type.append(al['album_type'])
        al_artists.append(al['artists'][0]['id'])
        al_external_urls.append(al['external_urls']['spotify'])
        al_id.append(al['id'])
        al_images.append(al['images'][0]['url'])
        al_name.append(al['name'])
        al_release_date.append(al['release_date'])
        al_uri.append(al['uri'])
    for i, t in enumerate(result['tracks']['items']):#les données des tracks
        t_album.append(t['album']['id'])
        t_artists.append(t['artists'][0]['id'])
        t_disc_number.append(t['disc_number'])
        t_duration_ms.append(t['duration_ms'])
        t_explicit.append(t['explicit'])
        t_external_urls.append(t['external_urls']['spotify'])
        t_id.append(t['id'])
        t_name.append(t['name'])
        t_popularity.append(t['popularity'])
        t_preview_url.append(t['preview_url'])
        t_number.append(t['track_number'])
        t_uri.append(t['uri'])

In [ ]:
# connexion a la base de donnée spotify_db
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-1MKAJNIV\MSSQLSERVER02;'
                      'Database=spotify_db;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

#Insertion des valeurs dans la base de donnée
for val1, val2, val3,val4, val5, val6 in zip (a_external_urls, a_followers, a_id,a_name,a_popularity, a_uri):
    a_values = (val1, val2, val3,val4, val5, val6)
    cursor.execute("""INSERT INTO artist (a_external_urls, a_followers, a_id,a_name,a_popularity, a_uri)
                  VALUES (?,?,?,?,?,?)""",a_values)
    cursor.commit()

for valal1, valal2, valal3,valal4, valal5, valal6, valal7, valal8 in zip (al_type, al_artists,al_external_urls,al_id,al_images, al_name,al_release_date, al_uri):
    al_values = (valal1, valal2, valal3,valal4, valal5, valal6, valal7, valal8)
    cursor.execute("""INSERT INTO album (al_type, al_artists,al_external_urls,al_id,al_images, al_name,al_release_date, al_uri)
                  VALUES (?,?,?,?,?,?,?,?)""",al_values)
    cursor.commit()
for valt1, valt2, valt3,valt4, valt5, valt6, valt7, valt8, valt9, valt10, valt11, valt12 in zip (t_album,t_artists,t_disc_number,t_duration_ms,t_explicit,t_external_urls,t_id,t_name,t_popularity ,t_preview_url, t_number,t_uri):
    t_values = (valt1, valt2, valt3,valt4, valt5, valt6, valt7, valt8, valt9, valt10, valt11, valt12)
    print(t_values)
    cursor.execute("""INSERT INTO track (t_album,t_artists,t_disc_number,t_duration_ms,t_explicit,t_external_urls,t_id,t_name,t_popularity ,t_preview_url,t_number,t_uri)
                  VALUES (?,?,?,?,?,?,?,?,?,?,?,?)""",t_values)
    cursor.commit()
cursor = conn.close()